In [1]:
import os
import sys
import torch
data_base_dir = "./test_data/"
# data_base_dir = "/home/bydguikong/yy_ws/dataset_generator/test_data/"
agent_position_data = torch.load(data_base_dir+"agent_position_data.pt")
agent_heading_data = torch.load(data_base_dir+"agent_heading_data.pt")
agent_velocity_data = torch.load(data_base_dir+"agent_velocity_data.pt")
agent_shape_data = torch.load(data_base_dir+"agent_shape_data.pt")
agent_category_data = torch.load(data_base_dir+"agent_category_data.pt")
agent_mask_data = torch.load(data_base_dir+"agent_mask_data.pt")

map_point_position_data = torch.load(data_base_dir+"map_point_position_data.pt")
map_point_vector_data = torch.load(data_base_dir+"map_point_vector_data.pt")
map_point_orientation_data = torch.load(data_base_dir+"map_point_orientation_data.pt")
map_polygon_center_data = torch.load(data_base_dir+"map_polygon_center_data.pt")
map_polygon_type_data = torch.load(data_base_dir+"map_polygon_type_data.pt")
map_polygon_on_route_data = torch.load(data_base_dir+"map_polygon_on_route_data.pt")
map_polygon_tl_status_data = torch.load(data_base_dir+"map_polygon_tl_status_data.pt")
map_polygon_has_speed_limit_data = torch.load(data_base_dir+"map_polygon_has_speed_limit_data.pt")
map_polygon_speed_limit_data = torch.load(data_base_dir+"map_polygon_speed_limit_data.pt")
map_mask_data = torch.load(data_base_dir+"map_mask_data.pt")

reference_line_position_data = torch.load(data_base_dir+"reference_line_position_data.pt")
reference_line_vector_data = torch.load(data_base_dir+"reference_line_vector_data.pt")
reference_line_orientation_data = torch.load(data_base_dir+"reference_line_orientation_data.pt")
reference_line_valid_mask_data = torch.load(data_base_dir+"reference_line_valid_mask_data.pt")

static_objects_position_data = torch.load(data_base_dir+"static_objects_position_data.pt")
static_objects_heading_data = torch.load(data_base_dir+"static_objects_heading_data.pt")
static_objects_shape_data = torch.load(data_base_dir+"static_objects_shape_data.pt")
static_objects_category_data = torch.load(data_base_dir+"static_objects_category_data.pt")
static_objects_valid_mask_data = torch.load(data_base_dir+"static_objects_valid_mask_data.pt")

current_state_data = torch.load(data_base_dir+"current_state_data.pt")

# candidate_trajectories_data = torch.load("./test_data/candidate_trajectories_data.pt")
# print("static_objects.position shape:", static_objects_position_data.shape)
# probability_data = torch.load("./test_data/probability_data.pt")


In [2]:
import onnx

# 获取当前文件所在的目录
current_directory = os.getcwd()

# print(f"current_directory:{current_directory}")
# ommx_model_pth = "/home/bydguikong/yy_ws/PlanScope/onnx/simplified_model.onnx"
ommx_model_pth = current_directory + "/diffusion_planner.onnx"

model = onnx.load(ommx_model_pth)
inferred_model = onnx.shape_inference.infer_shapes(model)
onnx.checker.check_model(inferred_model)

In [3]:
import onnxruntime as ort
import numpy as np

# 加载ONNX模型（请确保模型文件路径正确）
print("开始加载")
# session = ort.InferenceSession("/home/bydguikong/yy_ws/PlanScope/onnx/model.onnx")
so = ort.SessionOptions()
# so.log_severity_level = 0
so.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL  # 或者尝试其他级别

开始加载


In [4]:
session = ort.InferenceSession(ommx_model_pth, sess_options=so,
                               providers=['CPUExecutionProvider'])
print("加载成功")


加载成功


In [5]:
import torch
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")
# 获取模型的输入名称和形状
input_names = [
    "agent.position", "agent.heading", "agent.velocity", "agent.shape",
    "agent.category", "agent.valid_mask", #"agent.target",
    "map.point_position", "map.point_vector", "map.point_orientation", #"map.point_side",
    "map.polygon_center", #"map.polygon_position", #"map.polygon_orientation",
    "map.polygon_type", "map.polygon_on_route", "map.polygon_tl_status",
    "map.polygon_has_speed_limit", "map.polygon_speed_limit", #"map.polygon_road_block_id",
    "map.valid_mask",
    "reference_line.position", "reference_line.vector", "reference_line.orientation",
    "reference_line.valid_mask", #"reference_line.future_projection",
    "static_objects.position", "static_objects.heading", "static_objects.shape",
    "static_objects.category", "static_objects.valid_mask",
    "current_state", #"origin", "angle"
]

# for inp in session.get_inputs():
#     print(f"Input name: {inp.name}, Input shape:{inp.shape}")



In [6]:
import plotly.graph_objects as go
import numpy as np
from plotly.colors import qualitative
# 定义地图车道颜色组，每组包含中心线、左边界和右边界的颜色
color_pairs = [
    {"center": "#2C3E50", "left": "#1f77b4", "right": "#d62728"},  # 组0：中心线带蓝色调
    {"center": "#34495E", "left": "#17becf", "right": "#ff7f0e"},  # 组1：中心线带蓝灰调
    {"center": "#8E44AD", "left": "#0074D9", "right": "#FF4136"},  # 组2：中心线带紫色调
    {"center": "#16A085", "left": "#2E86C1", "right": "#E74C3C"},  # 组3：中心线带青绿色调
    {"center": "#27AE60", "left": "#2980B9", "right": "#E67E22"},  # 组4：中心线带绿色调
    {"center": "#D35400", "left": "#3498DB", "right": "#F39C12"},  # 组5：中心线带橙色调
    {"center": "#C0392B", "left": "#5DADE2", "right": "#E74C3C"},  # 组6：中心线带红色调
]

# 计算所有帧中 agent 数量的最大值
max_A = max([agent_position_item.shape[0] for agent_position_item in agent_position_data[0]])

# 使用 Plotly 提供的定性配色方案作为 agent 的颜色池（不足则循环使用）
agent_colors_pool = qualitative.Plotly  
agent_colors = [agent_colors_pool[i % len(agent_colors_pool)] for i in range(max_A)]

frames = []
n_frames = len(agent_position_data)
print(f"共{n_frames}帧")
step = max(n_frames//5,1)
print(f"step = {step}")
for frame_idx, (
    agent_position,
    agent_heading,
    agent_velocity,
    agent_shape,
    agent_category,
    agent_valid_mask,
    map_point_position,
    map_point_vector,
    map_point_orientation,
    map_polygon_center,
    map_polygon_type,
    map_polygon_on_route,
    map_polygon_tl_status,
    map_polygon_has_speed_limit,
    map_polygon_speed_limit,
    map_valid_mask,
    reference_line_position,
    reference_line_vector,
    reference_line_orientation,
    reference_line_valid_mask,
    static_objects_position,
    static_objects_heading,
    static_objects_shape,
    static_objects_category,
    static_objects_valid_mask,
    current_state,
    # candidate_trajectories,
    # probability
) in enumerate(zip(
    agent_position_data,
    agent_heading_data,
    agent_velocity_data,
    agent_shape_data,
    agent_category_data,
    agent_mask_data,
    map_point_position_data,
    map_point_vector_data,
    map_point_orientation_data,
    map_polygon_center_data,
    map_polygon_type_data,
    map_polygon_on_route_data,
    map_polygon_tl_status_data,
    map_polygon_has_speed_limit_data,
    map_polygon_speed_limit_data,
    map_mask_data,
    reference_line_position_data,
    reference_line_vector_data,
    reference_line_orientation_data,
    reference_line_valid_mask_data,
    static_objects_position_data,
    static_objects_heading_data,
    static_objects_shape_data,
    static_objects_category_data,
    static_objects_valid_mask_data,
    current_state_data,
    # candidate_trajectories_data,
    # probability_data
)):
    if frame_idx % step != 0:
        continue
    input_data = [
        agent_position.to(device),
        agent_heading.to(device),
        agent_velocity.to(device),
        agent_shape.to(device),
        agent_category.to(device),
        agent_valid_mask.to(device),
        map_point_position.to(device),
        map_point_vector.to(device),
        map_point_orientation.to(device),
        map_polygon_center.to(device),
        map_polygon_type.to(device),
        map_polygon_on_route.to(device),
        map_polygon_tl_status.to(device),
        map_polygon_has_speed_limit.to(device),
        map_polygon_speed_limit.to(device),
        map_valid_mask.to(device),
        reference_line_position.to(device),
        reference_line_vector.to(device),
        reference_line_orientation.to(device),
        reference_line_valid_mask.to(device),
        static_objects_position.to(device),
        static_objects_heading.to(device),
        static_objects_shape.to(device),
        static_objects_category.to(device),
        static_objects_valid_mask.to(device),
        current_state.to(device)
    ]
    # print(f"current_state.shape = {current_state.shape}")
    # print(f"current_state = {current_state[0]}")
    # print(f"agent0_position_list = {agent_position[0][0]}")
    # print(f"agent_category = {agent_category[0]}")
    # print(f"agent0_velocity = {agent_velocity[0][0]}")
    # print(f"agent0_heading = {agent_heading[0][0]}")
    # print(f"map_point_orientation = {map_point_orientation[0][0]}")
    # print(f"static_objects_position.shape = {static_objects_position.shape}")
    
    inputs = {name: tensor.cpu().numpy() for name, tensor in zip(input_names, input_data)}
    # for k,v in inputs.items():
    #     print(f"Input name: {k}, Input shape: {v.shape}, Input type: {v.dtype}")
    from to_diffusion_data import extract_model_inputs
    # 提取输入
    new_inputs = extract_model_inputs(inputs)
    print("---------------------------")
    for k, v in new_inputs.items():
        print(f"{k}: {v.shape}, dtype={v.dtype}")
    print("---------------------------")
    for input_meta in session.get_inputs():
        print(input_meta.name, input_meta.shape, input_meta.type)
    print("---------------------------")
    # ONNX 输入名顺序（必须和模型实际输入名完全对应）
    new_input_names = [
        "neighbor_agents_past",
        "ego_current_state",
        "static_objects",
        "lanes",
        "lanes_speed_limit",
        "lanes_has_speed_limit",
        "route_lanes",
        "route_lanes_speed_limit",
        "route_lanes_has_speed_limit",
    ]

    # 执行模型，输出名称一般为空列表或模型默认输出
    import time
    print("开始推理")

    start_time = time.time()
    output = session.run(None, new_inputs)  # None 表示返回所有输出
    # print(f"output.type = {type(output)}")
    # print(f"output.len = {len(output)}")
    print(f"output[0].shape = {output[0].shape}")
        
    map_item = map_point_position[0]      # shape: (D1, D2, P, 2)
    map_valid_item = map_valid_mask[0]            # shape: (D1, P)
    traces = []
    D1, D2, P, _ = map_item.shape
    bs, num_agents, num_history, _ = agent_position.shape
    # print(f"agent_position.shape = {agent_position.shape}")
    # 遍历每个车道
    for lane_idx in range(D1):
        color_set = color_pairs[lane_idx % len(color_pairs)]
        lane_mask = map_valid_item[lane_idx]  # 布尔张量
        lines = map_item[lane_idx]
        if D2 < 3:
            continue
        center_line = lines[0]
        left_bound = lines[1]
        right_bound = lines[2]

        center_valid = center_line[lane_mask]  # shape: (num_valid, 2)
        left_valid = left_bound[lane_mask]
        right_valid = right_bound[lane_mask]

        traces.append(go.Scatter(
            x=center_valid[:, 0].cpu().numpy().tolist() if center_valid.numel() > 0 else [],
            y=center_valid[:, 1].cpu().numpy().tolist() if center_valid.numel() > 0 else [],
            mode='lines',
            name=f"Lane {num_agents+lane_idx} Center",
            line=dict(color="gray", dash="dot"),
            showlegend=True,
        ))
        traces.append(go.Scatter(
            x=left_valid[:, 0].cpu().numpy().tolist() if left_valid.numel() > 0 else [],
            y=left_valid[:, 1].cpu().numpy().tolist() if left_valid.numel() > 0 else [],
            mode='lines',
            name=None,
            line=dict(color="gray"),
            showlegend=False,
        ))
        traces.append(go.Scatter(
            x=right_valid[:, 0].cpu().numpy().tolist() if right_valid.numel() > 0 else [],
            y=right_valid[:, 1].cpu().numpy().tolist() if right_valid.numel() > 0 else [],
            mode='lines',
            name=None,
            line=dict(color="gray"),
            showlegend=False,
        ))
    # print(f"agent_position.shape = {agent_position.shape}")
    # print(f"agent_heading.shape = {agent_heading.shape}")
    # print(f"agent_shape.shape = {agent_shape.shape}")
    # print(f"agent_valid_mask.shape = {agent_valid_mask.shape}")
    # 绘制智能体轨迹和安全盒
    # 数据格式说明：agent_position.shape = [bs, A, L, 2]
    #           agent_heading.shape  = [bs, A, L]
    #           agent_shape.shape    = [bs, A, L, 2] (长和宽)
    #           agent_valid_mask.shape = [bs, A, L]
    
    for agent_idx in range(num_agents):
        pos = agent_position[0, agent_idx].cpu()  # 新增 .cpu()
        heading = agent_heading[0, agent_idx].cpu()  # 新增 .cpu()
        shape_info = agent_shape[0, agent_idx].cpu()  # 新增 .cpu()
        valid_mask = agent_valid_mask[0, agent_idx].cpu()  # 新增 .cpu()

        valid_mask_bool = valid_mask.bool()
        valid_positions = pos[valid_mask_bool]
        if valid_positions.numel() > 0:
            traces.append(go.Scatter(
                x=valid_positions[:, 0].numpy().tolist(),
                y=valid_positions[:, 1].numpy().tolist(),
                mode='lines',
                name=None,
                line=dict(dash="solid", color="yellow"),
                showlegend=False,
            ))

        if valid_mask[-1]:
            last_pos = pos[-1]
            last_heading = heading[-1].item()
            last_shape = shape_info[-1]
            b_box = last_shape[0].item()
            l_box = last_shape[1].item()
            local_corners = np.array([
                [l_box / 2, b_box / 2],
                [l_box / 2, -b_box / 2],
                [-l_box / 2, -b_box / 2],
                [-l_box / 2, b_box / 2],
                [l_box / 2, b_box / 2]
            ])
            R = np.array([
                [np.cos(last_heading), -np.sin(last_heading)],
                [np.sin(last_heading), np.cos(last_heading)]
            ])
            rotated_corners = local_corners.dot(R.T)
            corners = rotated_corners + last_pos.numpy()
            traces.append(go.Scatter(
                x=corners[:, 0].tolist(),
                y=corners[:, 1].tolist(),
                mode='lines',
                name=f"Agent {agent_idx} Safety Box",
                line=dict(color="black", width=2),
                showlegend=True,
            ))
    # 规划轨迹可视化
    # candidate_trajectories: [bs, R, M, L, 3]   (x, y, yaw)
    # probability: [bs, R, M]
    
 
    prediction = output[0][0,:,:,:2] # probability

    
    R, L, F = prediction.shape
    
    # 为不同的参考车道分配不同颜色（使用 Plotly 定性配色方案）
    ref_colors = qualitative.Plotly
    ref_colors = [ref_colors[i % len(ref_colors)] for i in range(R)]
    # 用于图例控制：每个参考车道只显示一次图例
    ref_legend_added = [False] * R
    
    A, L, F = prediction.shape
    for i in range(A):
        traj = prediction[i]  # shape: (L, 5)
        # 对每个轨迹，将连续的点对分段绘制，颜色由白色到蓝色渐变
        for j in range(L-1):
            # 计算颜色渐变因子：0时为白色，1时为蓝色
            factor = j / (L - 1)
            # 白色：RGB (255,255,255)，蓝色：RGB (0,0,255)
            r = 0
            g = int(round(255 * factor))
            b = int(round(255 * (1 - factor)))
            color_hex = f"#{r:02x}{g:02x}{b:02x}"
            # 分段坐标：当前点和下一点
            x_segment = [traj[j, 0], traj[j+1, 0]]
            y_segment = [traj[j, 1], traj[j+1, 1]]
            # 对于每个智能体的第一段，添加图例项
            if j == 0:
                seg_name = f"Motion Prediction Agent {i}"
            else:
                seg_name = None
            traces.append(go.Scatter(
                x=x_segment,
                y=y_segment,
                mode='lines',
                name=seg_name,
                line=dict(color=color_hex, width=0.6),
                showlegend=(seg_name is not None)
            ))

    # 绘制参考车道线，参考车道数据维度为 [bs, R, L, 2]
    # print("reference_line_position.shape = ",reference_line_position.shape)
    ref_line_np = reference_line_position[0].cpu().detach().numpy()  # shape: [R, L, 2]
    # for r in range(R):
    #     ref_line = ref_line_np[r]  # shape: [L, 2]
    #     x_ref = ref_line[:, 0]
    #     y_ref = ref_line[:, 1]
    #     traces.append(go.Scatter(
    #         x=x_ref.tolist(),
    #         y=y_ref.tolist(),
    #         mode='lines',
    #         name=f"Reference Line {r}",
    #         line=dict(color=ref_colors[r], width=3, dash="dash"),
    #         showlegend=True,
    #     ))
        
    # 创建 Plotly 图形，将所有车道线拼接显示
    fig = go.Figure(data=traces)
    fig.update_layout(
        title="Map Lanes, Agent Trajectories & Planning Paths",
        xaxis_title="X",
        yaxis_title="Y",
        yaxis=dict(scaleanchor="x", scaleratio=1),
        width=700,
        height=500,
    )
    fig.show()
    
    # break

共298帧
step = 59
raw_inputs.agent.position:(1, 7, 21, 2)
raw_inputs.agent.heading:(1, 7, 21)
raw_inputs.agent.velocity:(1, 7, 21, 2)
raw_inputs.agent.shape:(1, 7, 21, 2)
raw_inputs.agent.category:(1, 7)
raw_inputs.agent.valid_mask:(1, 7, 21)
raw_inputs.map.point_position:(1, 40, 3, 20, 2)
raw_inputs.map.point_vector:(1, 40, 3, 20, 2)
raw_inputs.map.point_orientation:(1, 40, 3, 20)
raw_inputs.map.polygon_center:(1, 40, 3)
raw_inputs.map.polygon_type:(1, 40)
raw_inputs.map.polygon_on_route:(1, 40)
raw_inputs.map.polygon_tl_status:(1, 40)
raw_inputs.map.polygon_has_speed_limit:(1, 40)
raw_inputs.map.polygon_speed_limit:(1, 40)
raw_inputs.map.valid_mask:(1, 40, 20)
raw_inputs.reference_line.position:(1, 1, 80, 2)
raw_inputs.reference_line.vector:(1, 1, 80, 2)
raw_inputs.reference_line.orientation:(1, 1, 80)
raw_inputs.reference_line.valid_mask:(1, 1, 80)
raw_inputs.static_objects.position:(1, 3, 2)
raw_inputs.static_objects.heading:(1, 3)
raw_inputs.static_objects.shape:(1, 3, 2)
raw_inputs

raw_inputs.agent.position:(1, 8, 21, 2)
raw_inputs.agent.heading:(1, 8, 21)
raw_inputs.agent.velocity:(1, 8, 21, 2)
raw_inputs.agent.shape:(1, 8, 21, 2)
raw_inputs.agent.category:(1, 8)
raw_inputs.agent.valid_mask:(1, 8, 21)
raw_inputs.map.point_position:(1, 41, 3, 20, 2)
raw_inputs.map.point_vector:(1, 41, 3, 20, 2)
raw_inputs.map.point_orientation:(1, 41, 3, 20)
raw_inputs.map.polygon_center:(1, 41, 3)
raw_inputs.map.polygon_type:(1, 41)
raw_inputs.map.polygon_on_route:(1, 41)
raw_inputs.map.polygon_tl_status:(1, 41)
raw_inputs.map.polygon_has_speed_limit:(1, 41)
raw_inputs.map.polygon_speed_limit:(1, 41)
raw_inputs.map.valid_mask:(1, 41, 20)
raw_inputs.reference_line.position:(1, 1, 80, 2)
raw_inputs.reference_line.vector:(1, 1, 80, 2)
raw_inputs.reference_line.orientation:(1, 1, 80)
raw_inputs.reference_line.valid_mask:(1, 1, 80)
raw_inputs.static_objects.position:(1, 6, 2)
raw_inputs.static_objects.heading:(1, 6)
raw_inputs.static_objects.shape:(1, 6, 2)
raw_inputs.static_objects.

ValueError: could not broadcast input array from shape (41,) into shape (40,)